Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
import numpy as np
from icecream import ic

In [495]:
class Solution:

    def __init__(self, n_knapsacks, n_items, n_dimensions, values, weights, constraints, rng):
        self.n_knapsacks = n_knapsacks
        self.n_items = n_items
        self.n_dimensions = n_dimensions
        self.values = values
        self.weights = weights
        self.constraints = constraints
        self.rng = rng

    def is_valid(self, solution):
        # Check that no item appears in multiple knapsacks
        if not np.all(solution.sum(axis=0) <= 1):
            return False

        # Check constraint violations for each knapsack
        for k in range(self.n_knapsacks):
            items_in_knapsack = solution[k]
            
            # Only check if knapsack has items
            if np.any(items_in_knapsack):
                total_weight = self.weights[items_in_knapsack].sum(axis=0)
                if np.any(total_weight > self.constraints[k]):
                    return False

        return True

    def fitness(self, solution):
        #we assume the solution to be already validated, we do not call is_valid in this function
        return np.sum(self.values*solution) #total value of items in all knapsacks

    def tweak(self, solution):
        #idea: pick a random knapsack and item, flip their state and repeat the process n times. n is correlated to the ratio between the  assigned items and the total ones. If the ratio is low, a lot of items still have to be assigned, so we pick more knapsack and more items. Otherwise, less elements are selected
        tot_assigned = solution.sum()
        ratio = tot_assigned /self.n_items

        if ratio < 0.1:
            n = self.rng.integers(3,8)
        elif ratio < 0.3:
            n = self.rng.integers(2,5)
        else:
            n = self.rng.integers(1,3)


        for _ in range(n):
            k = self.rng.integers(0, self.n_knapsacks) #select a knapsack
            i = self.rng.integers(0, self.n_items)    #select an item
            
            if solution[k,i]:
                #item is in this knapsack --> remove it
                solution[k,i] = False
            else:
                if not np.any(solution[:,i]): #solution not in any other knapsack
                    solution[k,i] = True
                else: #is in another knapsack
                    current_knapsack = np.where(solution[:,i])[0][0] #get the knapsack where the item is currently in
                    solution[current_knapsack, i] = False #remove from this knapsack
                    solution[k,i] = True #add to new knapsack
            
        return solution
    


    def make_valid(self, solution):

        #try to make the solution valid (non valid solution are allowed, either for the initial state or intermiediate one)
        while(not self.is_valid(solution)):
            duplicates = np.where(solution.sum(axis=0)>1)[0] #for initial state, try to remove duplicates
            if len(duplicates) > 0:
                for item in duplicates:
                    knapsack_w_i = np.where(solution[:, item])[0] #get all knapsacks that have the duplicated item
                    for k in knapsack_w_i[1:]: #remove the item from all the other knapsacks
                        solution[k, item] = False
                    
            violating_k = []

            for k in range (self.n_knapsacks):  #try to find all the knapsacks that are violating constraints
                if np.any(self.weights[solution[k]].sum(axis = 0) > self.constraints[k]):
                    violating_k.append(k)

            if not violating_k:
                break
            
            for k in violating_k:
                num_items = np.where(solution[k] == True)[0] #get items that are in the knapsack k

                if len(num_items )> 0:
                    n_items_to_remove = self.rng.integers(1, len(num_items)) #select a random number of items to remove from knapsack that violates constraints
                    items_to_remove = self.rng.choice(num_items, size = n_items_to_remove, replace = False)

                    for i in items_to_remove:
                        solution[k, i] = False #remove items from knapsack

        return solution
    
    def resolve(self, max_iter = 1000):
        solution = np.array([np.random.random(self.n_items) < 0.5 for _ in range(self.n_knapsacks)], dtype=bool) #random sol
        
        solution = self.make_valid(solution)
        
        best_fitness = self.fitness(solution)
        best_solution = solution.copy()
        

        for _ in range(max_iter):
            current_sol = self.make_valid(self.tweak(best_solution.copy()))
            
            current_fitness = self.fitness(current_sol)

            if(current_fitness > best_fitness):
                best_solution = current_sol.copy()
                best_fitness = current_fitness
                ic(best_fitness)

        return best_solution, best_fitness


        


## TEST PROBLEMS

In [496]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [497]:
solver = Solution(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS, rng)
solution, fitness = solver.resolve(max_iter=10000)
solver.is_valid(solution)


ic| best_fitness: np.int64(603)
ic| best_fitness: np.int64(676)
ic| best_fitness: np.int64(773)
ic| best_fitness: np.int64(816)
ic| best_fitness: np.int64(892)
ic| best_fitness: np.int64(901)
ic| best_fitness: np.int64(921)
ic| best_fitness: np.int64(929)
ic| best_fitness: np.int64(1000)
ic| best_fitness: np.int64(1012)
ic| best_fitness: np.int64(1020)
ic| best_fitness: np.int64(1045)
ic| best_fitness: np.int64(1048)
ic| best_fitness: np.int64(1056)


True

In [498]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [499]:
solver = Solution(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS, rng)
solution, fitness = solver.resolve(max_iter=25000)
solver.is_valid(solution)

ic| best_fitness: np.int64(15096)
ic| best_fitness: np.int64(16454)
ic| best_fitness: np.int64(17460)
ic| best_fitness: np.int64(18096)
ic| best_fitness: np.int64(19041)
ic| best_fitness: np.int64(19267)
ic| best_fitness: np.int64(19736)
ic| best_fitness: np.int64(20494)
ic| best_fitness: np.int64(20857)
ic| best_fitness: np.int64(21414)
ic| best_fitness: np.int64(22195)
ic| best_fitness: np.int64(23117)
ic| best_fitness: np.int64(24595)
ic| best_fitness: np.int64(25453)
ic| best_fitness: np.int64(26008)
ic| best_fitness: np.int64(26453)
ic| best_fitness: np.int64(26863)
ic| best_fitness: np.int64(27057)
ic| best_fitness: np.int64(28073)
ic| best_fitness: np.int64(28806)
ic| best_fitness: np.int64(28898)
ic| best_fitness: np.int64(28975)
ic| best_fitness: np.int64(29430)
ic| best_fitness: np.int64(29742)
ic| best_fitness: np.int64(29785)
ic| best_fitness: np.int64(30626)
ic| best_fitness: np.int64(30827)
ic| best_fitness: np.int64(31895)
ic| best_fitness: np.int64(32690)
ic| best_fitne

True

In [500]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [501]:
solver = Solution(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS, rng)
solution, fitness = solver.resolve(max_iter=50000)
solver.is_valid(solution)

ic| best_fitness: np.int64(64794)
ic| best_fitness: np.int64(67027)
ic| best_fitness: np.int64(69503)
ic| best_fitness: np.int64(72115)
ic| best_fitness: np.int64(74630)
ic| best_fitness: np.int64(78953)
ic| best_fitness: np.int64(81416)
ic| best_fitness: np.int64(84191)
ic| best_fitness: np.int64(86979)
ic| best_fitness: np.int64(89807)
ic| best_fitness: np.int64(91548)
ic| best_fitness: np.int64(94104)
ic| best_fitness: np.int64(95837)
ic| best_fitness: np.int64(98424)
ic| best_fitness: np.int64(99787)
ic| best_fitness: np.int64(101836)
ic| best_fitness: np.int64(103513)
ic| best_fitness: np.int64(105168)
ic| best_fitness: np.int64(108357)
ic| best_fitness: np.int64(110581)
ic| best_fitness: np.int64(112928)
ic| best_fitness: np.int64(115906)
ic| best_fitness: np.int64(117647)
ic| best_fitness: np.int64(119943)
ic| best_fitness: np.int64(124387)
ic| best_fitness: np.int64(127981)
ic| best_fitness: np.int64(131039)
ic| best_fitness: np.int64(132645)
ic| best_fitness: np.int64(136318)


True